# Document Q&A with RAG using Chroma

For this project we are using Gemini API to create a vector database, retrieve answers to questions from the database and generate final answers.

Essentially RAG - uses Indexing, retrieval and Generation. Indexing is the ferrari, it happens ahead of time and lets us quickly loop up relevant infomration during query time. When query comes in, we retrieve revelant documents, combine them with predefined instruction along with use user's query, and then pass it on to LLM to generate a tailored answer. (This helps us to provide information to LLM that it hasn't seen before)

For vector database, we will use chroma, which is open sourced. With Chroma we will store embeddings alongside any metadeta, embed documents and queries, and finally search the documents.

Installing ChromaDB and the Gemini API Python SDK.

In [1]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown

In [3]:
genai.__version__

'1.7.0'

### Setting API key

Using Kaggle Secret to set up Gemini API key

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

### Models that support Embeddings

We will be using the [`embedContent`] API method to calculate embeddings. Models that supports it can be found using the [`models.list`] endpoint.

We will use `text-embedding-004` which is the most recent available embedding model.

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


### Sample Data

Here is a small set of documents we will use to create an embedding database.

In [6]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## Creating the embedding database with ChromaDB

Example of a [custom function](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) to generate embeddings with the Gemini API. 

Here, I am implementing a retrieval system, so the `task_type` for generating the *document* embeddings is `retrieval_document`. Later, we will use `retrieval_query` for the *query* embeddings. 

Other [API reference](https://ai.google.dev/api/embeddings#v1beta.TaskType) for similar but different tasks.

In [7]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# This will generate embeddings for both documents and queries.
class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Now we create a [Chroma database client](https://docs.trychroma.com/getting-started) that uses the `GeminiEmbeddingFunction` and populate the database with the documents you defined above.

In [8]:
import chromadb

DB_NAME = "google_collection_db"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

#Each document will have embedding based on it's numbering.
db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

Confirm that the data was inserted by looking at the database.

In [9]:
db.count()

3

## Retrieval: Find relevant documents

To search the Chroma database, call the `query` method. We will also switch to `retrieval_query` mode of embedding generation.

`n_result` is the number of results to return from chromadb


In [10]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Can I play music on Google Car?"

result = db.query(query_texts=[query], n_results=1)

print(result["documents"][0]) ## This will be a list with n_result number of document.

['Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.']


## From Generation to Augmented generation

Now that we have found a relevant passage from the set of documents (the *retrieval* step), we will now assemble a generation prompt to have the Gemini API *generate* a final answer.

In [11]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in result["documents"][0]:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: Can I play music on Google Car?
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



Now use the `generate_content` method to to generate an answer to the question.

In [12]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

answer.text

'Yes, your Googlecar is equipped with a touchscreen display that gives you access to many features, and to play your favorite songs, you can simply touch the "Music" icon on the display.\n'